In [2]:
import math
import csv
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import choice


from io import StringIO

length = 500

In [3]:
char_to_index_map = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7,
    'i': 8,
    'j': 9,
    'k': 10,
    'l': 11,
    'm': 12,
    'n': 13,
    'o': 14,
    'p': 15,
    'q': 16,
    'r': 17,
    's': 18,
    't': 19,
    'u': 20,
    'v': 21,
    'w': 22,
    'x': 23,
    'y': 24,
    'z': 25,
    ',': 26,
    '_': 27,
    '?': 28,
    """'""": 29,
    '\\': 30,
    '/': 31,
}

index_to_char_map = {
    0: 'a',
    1: 'b',
    2: 'c',
    3: 'd',
    4: 'e',
    5: 'f',
    6: 'g',
    7: 'h',
    8: 'i',
    9: 'j',
    10: 'k',
    11: 'l',
    12: 'm',
    13: 'n',
    14: 'o',
    15: 'p',
    16: 'q',
    17: 'r',
    18: 's',
    19: 't',
    20: 'u',
    21: 'v',
    22: 'w',
    23: 'x',
    24: 'y',
    25: 'z',
    26: ',',
    27: '_',
    28: '?',
    29: """'""",
    30: '\\',
    31: '/',
}

base_array = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                       0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                       0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                       0, 0
                       ])


In [4]:
def load_lyrics():
    lyrics = ''
    data = open('lyric2.txt', 'r')
    for i in data:
    	lyrics += i
    return lyrics

def print_lyrics(data):
    for value in data:
        print(value)

def convert_char_to_array(character):
    result = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                       0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                       0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                       0, 0
                      ])
    index = char_to_index_map[character]
    result[index] = 1
    return result

def generate_input_data(all_chars):
    input_data = []
    for character in all_chars:
        new_array = convert_char_to_array(character)
        input_data.append(new_array)
    return input_data

def generate_teacher_data(all_chars):
    output_data = convert_char_to_array(all_chars[1])
    for character in all_chars[2:len(all_chars)]:
        new_array = convert_char_to_array(character)
        output_data = np.vstack((output_data, new_array))
    output_data = np.vstack((output_data, base_array))
    return output_data

def generate_data(all_chars):
    input_data = generate_input_data(all_chars)
    #print('Input data generated.')
    teacher_data = generate_teacher_data(all_chars)
    #print('Teacher data generated.')

    return input_data, teacher_data

def create_vector_by_index(index):
    result = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                       0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                       0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                       0, 0, 0, 0, 0, 0, 0
                       ])
    result[index] = 1
    return result

def decrypt_output(collection_matrix):
    description = ''
    for i in range(length + 1):
        letter_vector = collection_matrix[:, i]
        letter = choose_letter(letter_vector)
        description = '{}{}'.format(description, letter)
    print(description)

def choose_letter(input_vector):
    #chosen_index = np.argmax(input_vector)
    input_vector = np.array(input_vector)
    #print(input_vector.sum())
    chosen_index = np.random.choice(32, 1, replace=False, p=input_vector)
    letter = index_to_char_map[chosen_index[0]]
    return letter

def normalize_vector(input_vector):
    elements_sum = 0
    result_vector = np.array(input_vector)
    for i in range(input_vector.size):
        if input_vector[i] < 0:
            result_vector[i] = 0
        else:
            result_vector[i] = input_vector[i]

    sum_of_elements = result_vector.sum()
    result_vector = result_vector/result_vector.sum()
    #for i in range(result_vector.size):
    #    result_vector[i] = result_vector[i] / sum_of_elements
    #    suma += result_vector[i]
    suma=result_vector.sum()
    #print(suma)
    return result_vector

In [15]:
if __name__ == '__main__':

    all_lyrics = load_lyrics()
    #print("lyrics loaded")
    #print_lyrics(all_lyrics)
    all_lyrics = all_lyrics.lower()
    input_data, teacher_data = generate_data(all_lyrics)
    #print('Input data and teacher data loaded ...')

    input_units = output_units = 32
    units = 400
    np.random.seed(78)
    # Define the reservoir parameters
    defined = 1

    if defined == 0:
        Win = (np.random.rand(units, input_units) - 0.5)
        W = np.random.rand(units, units) - 0.5
        bias = np.random.rand(units, ) - 0.5
        np.savetxt('Win.txt', Win)
        np.savetxt('W.txt', W)
        np.savetxt('bias.txt', bias)
    read = 1
    if read == 1:
        Win = np.genfromtxt('Win.txt')
        W = np.genfromtxt('W.txt')
        bias = np.genfromtxt('bias.txt')

    spectral_radius = np.max(np.abs(np.linalg.eig(W)[0]))
    #print('Spectral radius is ...', spectral_radius)
    W *= 1 / spectral_radius

    winsf = 1.6
    wsf = 1.4
    bsf = 0.9
    alpha = 0.005
    
    W = np.multiply(W, wsf)
    Win = np.multiply(Win, winsf)
    bias = np.multiply(bias, bsf)

    train_len = len(input_data)
    sum_of_entropy_rates = 0

    # Data collection matrix
    # State collection matrix
    X = np.zeros((input_units + units, train_len), dtype=np.float32)
    x = np.zeros((units, ), dtype=np.float32)

    #print('Starting Optimizer...')
    folds = 5
    opt_len = train_len - math.floor(train_len/folds) - 1

    for i in range(folds):
        a = (i)*(math.floor(train_len/folds))
        b = (i+1)*(math.floor(train_len/folds))
        input_data = np.array(input_data)
        U = np.concatenate((input_data[0:a], input_data[(b):(train_len-1)]),axis=0)
        testU = np.array(input_data[(a): b]); #validation set for input data
        testD = np.array(input_data[(a+1): b+1]); #validation set for teacher data
        D = np.concatenate((input_data[1:a+1], input_data[(b+1):(train_len)]), axis=0) #testing teacher data
        
        #print('Starting training in ' , i , 'fold')
        X_train = np.zeros((units+input_units, U.shape[0]), dtype=np.float32)
        for i2 in range(opt_len-1):
            u = U[i2]
            x = np.tanh(np.dot(W, x) + np.dot(Win, u) + bias)
            collection_vector = np.vstack((u[:, None], x[:, None]))
            X_train[:, i2] = collection_vector[:, 0]
        
           
        # Use Ridge regression to train the output matrix
        Wout = np.dot(np.divide(np.dot(D.T, X_train.T),train_len), np.linalg.inv(np.divide(np.dot(X_train, X_train.T), train_len) + alpha*np.eye(units + input_units)))
        
        test_len = math.floor(train_len/folds)
        nhv_test = np.zeros((output_units, testU.size), dtype=np.float32)
        X_test = np.zeros((units+input_units, testU.size), dtype=np.float32)
        sum_of_logs = 0
        loop_value = a
        value = 0
        #print('Starting testing in ' , i , 'fold')
        for i3 in range(test_len-1):
            u = testU[i3]
            x = np.tanh(np.dot(W, x) + np.dot(Win, u) + bias)
            collection_vector = np.vstack((u[:, None], x[:, None]))
            X_test[:, i3] = collection_vector[:, 0]
            y = np.dot(Wout, collection_vector[:, 0])
            y_normalized = normalize_vector(y)
            current_character = all_lyrics[loop_value]
            current_character_index = char_to_index_map[current_character]
            if (y_normalized[current_character_index] == 0):
                value = 0
            else:
                value = (np.log10(y_normalized[current_character_index]))
            sum_of_logs += value
            nhv_test[:, i3] = y_normalized
            loop_value += 1
        entropy_rate = sum_of_logs/train_len
        sum_of_entropy_rates += entropy_rate
    
    mean_entropy_rate = sum_of_entropy_rates/folds
    print(mean_entropy_rate)

-0.17599450459520108


In [30]:
test1 = np.zeros((12425, 32), dtype=np.float32)
np.concatenate((input_data[0:3105], input_data[6212:15530]), axis=0, out=test1)

ValueError: Output array is the wrong shape

In [131]:
a = [[1,2,3], [5,6,7], [4,5,6], [7,8,9], [4,6,8]]
B = np.concatenate((a[0:1], a[3:4]), axis=0)
print(B)

[[1 2 3]
 [7 8 9]]


In [6]:
print(W.shape)

(400, 400)
